In [3]:
from newspaper import Article #newspaper has some nlp stuff, don't know if it is good enough

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [4]:
#https://towardsdatascience.com/scrape-and-summarize-news-articles-in-5-lines-of-python-code-175f0e5c7dfc
url = "https://thenewstack.io/dont-mess-with-the-master-working-with-branches-in-git-and-github/"
article = Article(url, language='en', fetch_images=False)

#download and parse the article
article.download()
article.parse()
#article.nlp() #Don't know if we need to parse

text = article.text

In [5]:
doc = nlp(text)
#displacy.render(nlp(str(sentences[20])), jupyter=True, style='ent')

labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter({'ORG': 14,
         'GPE': 2,
         'CARDINAL': 11,
         'NORP': 4,
         'ORDINAL': 2,
         'WORK_OF_ART': 3,
         'PERSON': 1})

In [6]:
sentences = [x for x in doc.sents]
print(sentences[21])

Instead, everyone uses branches created from master to experiment, make edits and additions and changes, before eventually rolling that branch back into the master once they have been approved and are known to work.


In [23]:
items = [x.text for x in doc.ents]
Counter(items).most_common(3)

[('hello_octo', 10),
 ('GitHub', 9),
 ('one', 3),
 ('Github.com', 1),
 ('more than one', 1),
 ('One', 1),
 ('1', 1),
 ('first', 1),
 ('second', 1),
 ('2', 1),
 ('Hello World', 1),
 ('two', 1),
 ('git checkout hello_octo', 1),
 ('git checkout hello_kitty', 1),
 ('Git', 1),
 ('3', 1),
 ('4', 1),
 ('5', 1)]

In [20]:
#! python3 -m nltk.downloader wordnet
#! python3 -m nltk.downloader omw
#! pip3 install spacy-wordnet
from spacy_wordnet.wordnet_annotator import WordnetAnnotator

In [21]:
#nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')
for (item, count) in Counter(items).most_common():
    token = nlp(item)[0]
    print(token)
    print(token._.wordnet.synsets())
    print(token._.wordnet.lemmas())
    print(token._.wordnet.wordnet_domains())
    print()


hello_octo
[]
[]
[]

GitHub
[]
[]
[]

one
[Synset('one.n.01'), Synset('one.n.02'), Synset('one.s.01'), Synset('one.s.02'), Synset('one.s.03'), Synset('one.s.04'), Synset('one.s.05'), Synset('one.s.06'), Synset('matchless.s.01')]
[Lemma('one.n.01.one'), Lemma('one.n.01.1'), Lemma('one.n.01.I'), Lemma('one.n.01.ace'), Lemma('one.n.01.single'), Lemma('one.n.01.unity'), Lemma('one.n.02.one'), Lemma('one.s.01.one'), Lemma('one.s.01.1'), Lemma('one.s.01.i'), Lemma('one.s.01.ane'), Lemma('one.s.02.one'), Lemma('one.s.02.unitary'), Lemma('one.s.03.one'), Lemma('one.s.04.one'), Lemma('one.s.05.one'), Lemma('one.s.06.one'), Lemma('matchless.s.01.matchless'), Lemma('matchless.s.01.nonpareil'), Lemma('matchless.s.01.one'), Lemma('matchless.s.01.one_and_only'), Lemma('matchless.s.01.peerless'), Lemma('matchless.s.01.unmatched'), Lemma('matchless.s.01.unmatchable'), Lemma('matchless.s.01.unrivaled'), Lemma('matchless.s.01.unrivalled')]
['number', 'number', 'philosophy', 'philosophy', 'philosophy']



In [ ]:
#https://stackoverflow.com/questions/22031968/how-to-find-distance-between-two-synset-using-python-nltk-in-wordnet-hierarchy
# want to do this next